# Contents of This Notebook

1. Import necessary libraries and read data from files.
2. Preprocessing
3. Baseline Model
4. XGBoost Model with Time-Series Awareness

Note that exploratory data analysis is in eda.ipynb, done by teammate Xueying Ng.

## 1. Import necessary libraries and read data from files.

In [5]:
# Import necessary libraries
# BASE
# ------------------------------------------------------
import numpy as np
import pandas as pd
import os
import gc
import warnings

# PACF - ACF
# ------------------------------------------------------
import statsmodels.api as sm

# DATA VISUALIZATION
# ------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


# CONFIGURATIONS
# ------------------------------------------------------
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings('ignore')

In [6]:
import os
for dirname, _, filenames in os.walk('/Users/chloeyueh/Documents/git-repos/kaggle_rookies/competition-data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/Users/chloeyueh/Documents/git-repos/kaggle_rookies/competition-data/test.csv
/Users/chloeyueh/Documents/git-repos/kaggle_rookies/competition-data/train.csv
/Users/chloeyueh/Documents/git-repos/kaggle_rookies/competition-data/transactions.csv
/Users/chloeyueh/Documents/git-repos/kaggle_rookies/competition-data/oil.csv
/Users/chloeyueh/Documents/git-repos/kaggle_rookies/competition-data/holidays_events.csv
/Users/chloeyueh/Documents/git-repos/kaggle_rookies/competition-data/sample_submission.csv
/Users/chloeyueh/Documents/git-repos/kaggle_rookies/competition-data/stores.csv


In [7]:
# Read the data from all the files and keeping it for future use
train = pd.read_csv('/Users/chloeyueh/Documents/git-repos/kaggle_rookies/competition-data/train.csv')
test = pd.read_csv('/Users/chloeyueh/Documents/git-repos/kaggle_rookies/competition-data/test.csv')
stores = pd.read_csv('/Users/chloeyueh/Documents/git-repos/kaggle_rookies/competition-data/stores.csv')
transactions = pd.read_csv('/Users/chloeyueh/Documents/git-repos/kaggle_rookies/competition-data/transactions.csv')
oil = pd.read_csv('/Users/chloeyueh/Documents/git-repos/kaggle_rookies/competition-data/oil.csv')
holidays = pd.read_csv('/Users/chloeyueh/Documents/git-repos/kaggle_rookies/competition-data/holidays_events.csv')
sample = pd.read_csv('/Users/chloeyueh/Documents/git-repos/kaggle_rookies/competition-data/sample_submission.csv')

## 2. Preprocessing

This step involves processing training and testing data in the same way using <code>process_data</code> function, and actively making sure that the one-hot encoded columns match.

**Functions in this step**

<code>adjust_holidays</code> adjusts holidays according to the following steps:
- By default all rows are marked as holidays (is_real_holiday=1)
- If transferred is True but type is not 'Transfer', it is assumed that the date is not actually a holiday and is set to 0
- Set type 'Work Day' to 0 (non-holiday)

<code>process_data</code> integrates and preprocesses data:
- Filter data based on specified date range
- Merge stores, oil and holidays data into the main data set
- Forward filling of missing values ​​for crude oil price (assuming field name 'dcoilwtico')
- Create binary holiday token is_holiday based on holidays data, using adjusted is_real_holiday
- Added salary payment indicator (is_payday): If the date is the 15th or the end of the month, it will be marked as 1
- One-hot encoding of product families
    
**函式**

<code>adjust_holidays</code> 根據說明調整假日資料（<code>holidays</code>）：
- 預設所有行均標記為假日 (is_real_holiday=1)
- 如果 transferred 為 True，但 type 不是 'Transfer'，則認為該日期實際上不是慶祝假日，設為 0
- 將 type 為 'Work Day' 的設為 0（非假日）


<code>process_data</code> 整合並預處理數據（<code>train</code>、<code>test</code>）：
- 根據指定日期範圍過濾數據
- 合併 stores、oil 與 holidays 資料到主數據集
- 前向填充原油價格缺失值 (假設欄位名稱 'dcoilwtico')
- 根據 holidays 資料創建二進制假日標記 is_holiday，採用經過調整的 is_real_holiday
- 新增薪資發放指標 (is_payday)：若該日期為 15 號或月末，則標記為 1
- 對產品家族進行 one-hot 編碼

處理完的數據有以下欄位：
- 'id', 'date', 'store_nbr', 'sales', 'onpromotion', 'city', 'state', 'type', 'cluster', 'dcoilwtico', 'is_holiday', 'is_payday',
- 'family_AUTOMOTIVE', 'family_BABY CARE', 'family_BEAUTY', 'family_BEVERAGES', 'family_BOOKS', 'family_BREAD/BAKERY', 'family_CELEBRATION', 'family_CLEANING', 'family_DAIRY', 'family_DELI', 'family_EGGS', 'family_FROZEN FOODS', 'family_GROCERY I', 'family_GROCERY II', 'family_HARDWARE', 'family_HOME AND KITCHEN I', 'family_HOME AND KITCHEN II', 'family_HOME APPLIANCES', 'family_HOME CARE', 'family_LADIESWEAR', 'family_LAWN AND GARDEN', 'family_LINGERIE', 'family_LIQUOR,WINE,BEER', 'family_MAGAZINES', 'family_MEATS', 'family_PERSONAL CARE', 'family_PET SUPPLIES', 'family_PLAYERS AND ELECTRONICS', 'family_POULTRY', 'family_PREPARED FOODS', 'family_PRODUCE', 'family_SCHOOL AND OFFICE SUPPLIES', 'family_SEAFOOD'

In [ ]:
def adjust_holidays(holidays_df):
    holidays_df = holidays_df.copy()
    holidays_df['is_real_holiday'] = 1
    holidays_df.loc[(holidays_df['transferred'] == True) & (holidays_df['type'] != 'Transfer'), 'is_real_holiday'] = 0
    holidays_df.loc[holidays_df['type'] == 'Work Day', 'is_real_holiday'] = 0
    return holidays_df

def process_data(df, stores, oil, holidays, start_date=None, end_date=None):
    # 日期格式處理
    df['date'] = pd.to_datetime(df['date'])
    oil['date'] = pd.to_datetime(oil['date'])
    holidays['date'] = pd.to_datetime(holidays['date'])

    # 日期範圍過濾（只對訓練資料可能有意義）
    if start_date:
        df = df[df['date'] >= pd.to_datetime(start_date)]
    if end_date:
        df = df[df['date'] <= pd.to_datetime(end_date)]

    # 合併商店資訊
    df = pd.merge(df, stores, on='store_nbr', how='left')

    # 合併油價
    df = pd.merge(df, oil, on='date', how='left')
    if 'dcoilwtico' in df.columns:
        df['dcoilwtico'] = df['dcoilwtico'].fillna(method='ffill')

    # 假日處理
    holidays_adj = adjust_holidays(holidays)
    holidays_sel = holidays_adj[['date', 'is_real_holiday']]
    df = pd.merge(df, holidays_sel, on='date', how='left')
    df['is_real_holiday'] = df['is_real_holiday'].fillna(0)
    df.rename(columns={'is_real_holiday': 'is_holiday'}, inplace=True)

    # 薪資日（每月 15 號和月末）
    df['is_payday'] = ((df['date'].dt.day == 15) | (df['date'].dt.is_month_end)).astype(int)

    # One-hot 編碼 product family（先記住有哪些 family）
    df = pd.get_dummies(df, columns=['family'], prefix='family')

    return df

# 使用 process_data() 處理訓練數據，指定日期範圍（根據需要調整）
processed_train = process_data(train, stores, oil, holidays, start_date='2013-01-01', end_date='2017-08-15')
processed_test = process_data(test, stores, oil, holidays)

# Check dataframe columns -- not necessary
# print(pd.DataFrame(processed_train))
# print(pd.DataFrame(processed_test).columns)

              id       date  store_nbr   sales  onpromotion   city      state  \
0              0 2013-01-01          1    0.00            0  Quito  Pichincha   
1              1 2013-01-01          1    0.00            0  Quito  Pichincha   
2              2 2013-01-01          1    0.00            0  Quito  Pichincha   
3              3 2013-01-01          1    0.00            0  Quito  Pichincha   
4              4 2013-01-01          1    0.00            0  Quito  Pichincha   
...          ...        ...        ...     ...          ...    ...        ...   
3054343  3000883 2017-08-15          9  438.13            0  Quito  Pichincha   
3054344  3000884 2017-08-15          9  154.55            1  Quito  Pichincha   
3054345  3000885 2017-08-15          9 2419.73          148  Quito  Pichincha   
3054346  3000886 2017-08-15          9  121.00            8  Quito  Pichincha   
3054347  3000887 2017-08-15          9   16.00            0  Quito  Pichincha   

        type  cluster  dcoi

**Ensure One-Hot Columns Match** 

Make sure both processed_train and processed_test have the same family columns.

In [5]:
# Get all family columns from training set
train_fam_cols = [col for col in processed_train.columns if col.startswith('family_')]

# Get all family columns from test set
test_fam_cols = [col for col in processed_test.columns if col.startswith('family_')]

# Find missing columns in test
missing_in_test = list(set(train_fam_cols) - set(test_fam_cols))

# Add missing columns with 0s
for col in missing_in_test:
    processed_test[col] = 0

# Reorder test columns to match train and exclude sales column
processed_test = processed_test[processed_train.columns.drop('sales')]  # exclude sales


## 3. Set Up a Baseline Model (Mean Sales Per Store-Family)
This model will predict the average sales for each (store_nbr, family) pair in the training data. "For every store and product family, predict future sales as the average of its past sales."

In [6]:
# Group by store and family to get the mean sales
store_family_mean = train.groupby(['store_nbr', 'family'])['sales'].mean().reset_index()
store_family_mean.columns = ['store_nbr', 'family', 'mean_sales']

# Merge the mean with test data
baseline_preds = pd.merge(test, store_family_mean, on=['store_nbr', 'family'], how='left')

# Fill missing means with global mean if any (e.g., new store-family combos)
global_mean = train['sales'].mean()
baseline_preds['mean_sales'] = baseline_preds['mean_sales'].fillna(global_mean)

# Prepare submission
submission = baseline_preds[['id']].copy()
submission['sales'] = baseline_preds['mean_sales']

# Export to CSV
submission.to_csv('baseline_submission.csv', index=False)